In [ ]:
import os
from random import randint
from matplotlib import pyplot as plt
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from python_research.experiments.multiple_feature_learning.utils.utils import load_patches, combine_patches
from python_research.experiments.multiple_feature_learning.utils.dataset import Dataset
from python_research.experiments.multiple_feature_learning.builders.keras_builders import build_model, build_settings_for_dataset
from python_research.preprocessing.grids.grid_extraction import extract_grids
%matplotlib inline

PATCHES_DIRECTORY = ""
DATASET_PATH = "C:\\Users\mmyller.KPLABS\Documents\datasets\pavia\PaviaU_corrected.npy"
DATASET_GT_PATH = "C:\\Users\mmyller.KPLABS\Documents\datasets\pavia\PaviaU_gt.npy"
OUTPUT_PATH = "grids_validation_3D\\artifact"
PATCH_SIZE = (17, 30)
PIXEL_NEIGHBOURHOOD = (7, 7)
TOTAL_NUMBER_OF_TRAIN_SAMPLES = 2700
CLASSES_COUNT = 9
PATIENCE = 15
EPOCHS = 200
BATCH_SIZE = 64
os.makedirs("grids_validation_3D", exist_ok=True)


# Load data if path to the folder with patches is specified
if PATCHES_DIRECTORY != "":
    train_val_data, test_data = load_patches(PATCHES_DIRECTORY, PIXEL_NEIGHBOURHOOD, CLASSES_COUNT)
    dataset_image = test_data.x[:, :, randint(0, test_data.x.shape[-1])]
# Extract grids from provided dataset
else:
    patches, test_set, dataset_image = extract_grids(DATASET_PATH, DATASET_GT_PATH, PATCH_SIZE, 
                                                     TOTAL_NUMBER_OF_TRAIN_SAMPLES)
    train_val_data, test_data = combine_patches(patches[0], patches[1], test_set[0], test_set[1], 
                                                PIXEL_NEIGHBOURHOOD, CLASSES_COUNT)    
# Show location of extracted grids
plt.imshow(dataset_image)
plt.show()


In [ ]:

# Normalize data
train_val_data.normalize_data(CLASSES_COUNT)
# Normalize test data using min and max from train data
test_data.x_test = (test_data.x_test - train_val_data.min) / (train_val_data.max - train_val_data.min)

bands_count = test_data.x.shape[-1]
# Build model
settings = build_settings_for_dataset(PIXEL_NEIGHBOURHOOD)
model = build_model(settings, CLASSES_COUNT, bands_count)
print(model.summary())


In [ ]:
# Callbacks
early = EarlyStopping(patience=PATIENCE)
checkpoint = ModelCheckpoint(OUTPUT_PATH + "_model", save_best_only=True)

model.fit(x=train_val_data.x_train, y=train_val_data.y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=False,
          callbacks=[early, checkpoint], validation_data=[train_val_data.x_val, train_val_data.y_val])

best_model = load_model(OUTPUT_PATH + "_model")

# Evaluate test set score
accuracy = best_model.evaluate(x=test_data.x_test, y=test_data.y_test)[1]
print("Test set accuracy: {}".format(accuracy))
